In [1]:
#this code downloads ESAs from section 52 applications
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from IPython.core.interactiveshell import InteractiveShell

from urllib.request import urlretrieve
import os

os.getcwd()
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#folder with list of missing ESA folder
#df of missing projects
missing = pd.read_csv('F:/Environmental Baseline Data/Version 2/Support files/list_of_Section_52_projects.csv', 
                      encoding='ISO-8859-1')
missing.head()
missing.shape
missing['Application title'].nunique()

,Section,Application title,link to folder with ESA,Folder link,Company Name,Consultant Name
0,52,2003-03-17 Application to Construct and Operat...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,268693,EnCana Ekwan Pipeline Inc.,AXYS Environmental Consulting Ltd.
1,52,2006-05-23 - Application for the Brunswick Pip...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,408713,Emera Brunswick Pipeline Company Ltd.,Jacques Whitford
2,52,"2007-03-09 - Application for Line 13 Transfer,...",https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,456315,Enbridge Southern Lights GP Inc.,TERA Environmental Consultants
3,52,2007-06-28 - Application for the Line 4 Extens...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4...,467952,Enbridge Pipelines Inc.,TERA Environmental Consultants
4,52,2008-02-27 - Application to construct and oper...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/5...,500494,Westcoast Energy Inc.,TERA Environmental Consultants


(67, 6)

36

In [3]:
#list of links 
urls_list = missing['link to folder with ESA'].tolist()
urls_list_load = [str(x).replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/View/', 
                                 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/') for x in urls_list]

In [4]:
all_esa = []

for x in urls_list_load:
    cookies = {'RDI-NumberOfRecords': '200'}
    r = requests.get(x, cookies = cookies)
    soup = bs(r.text, 'lxml')
    tds = soup.find_all('td', {'class' : 'nopadding break-anywhere'})
    file_name = [x.text for x in soup.find_all('a', {'class' : 'row-icon'})]
    hrefs = [a['href'] for a in soup.find_all('a', {'class' : 'row-icon'})]
    df = pd.DataFrame({'file_name' : file_name, 'hrefs' : hrefs, 'folder' : str(x)})
    all_esa.append(df)

In [5]:

df = pd.concat(all_esa, axis = 0)
df = df[df['file_name'].str.contains('ESA|nvironment|Horn|ocio|EIA|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|A1X1|A1C3|A3S1|A3S2|Goldboro')]
df['Folder link'] = df['folder'].str.replace('https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/', '').astype(int)
df = df.merge(missing, on = 'Folder link')
postal_series = df['file_name'].str.extract('(A\d\D\d\D\d)')
df.insert(1, 'postal', postal_series)

###
href_id = df['hrefs'].str.rsplit('/' , 1).str[1]
df.insert(1, 'hrefs_id', href_id)
###

# df['Folder link'] = df['Folder link']
df.shape

(1331, 11)

In [6]:
df_ = df.drop_duplicates('hrefs_id').reset_index(drop=True)

In [7]:
df_.shape

(1328, 11)

In [8]:

#href_id = df['hrefs'].str.split('/' , n= 4, expand=True)
#href_id = df['hrefs'].str.rsplit('/' , 1).str[1]

#df.insert(1, 'href_id', href_id)
#href_id

In [9]:
#check to ensure all ESPs are captured; should return an empty; if project name, ESAs not captured
df_list = df['Application title'].unique()
missing_list = missing['Application title'].unique()

print(list(set(missing_list) - set(df_list)))


[]


In [10]:
# leave for future in case need to take Projects out of corpus
# remove three projects b/c they already exist in original corpus (ie. ESA data is already in corpus)
# searchfor = ['Groundbirch', 'Towerbirch', 'GH-001-2012']
# df = df[~df['Application title'].str.contains('|'.join(searchfor))]
# df.shape

In [11]:
#clean up href link


df_['hrefs'] = 'https://apps.cer-rec.gc.ca/REGDOCS/File/Download/' + df_['hrefs_id'].astype(str)
df_['dir_pdf'] = 'F:/Environmental Baseline Data/Version 3/Data/PDF/' + df_['postal'].astype(str)+'.pdf'
download_ESA_links = df_['hrefs']
download_ESA_filename = df_['postal']
# assing the href_id as file names
len(download_ESA_links)
len(download_ESA_filename)
df_.columns
df_.hrefs[0]
df_.dir_pdf[0]

1328

1328

Index(['file_name', 'hrefs_id', 'postal', 'hrefs', 'folder', 'Folder link',
       'Section', 'Application title', 'link to folder with ESA',
       'Company Name', 'Consultant Name', 'dir_pdf'],
      dtype='object')

'https://apps.cer-rec.gc.ca/REGDOCS/File/Download/268706'

'F:/Environmental Baseline Data/Version 3/Data/PDF/A0H8C0.pdf'

In [12]:
df_['file_name'].nunique()

1328

In [13]:
#df.to_excel('esas.xlsx')
#df_.to_excel('esas_no_duplicates_corrected.xlsx')

In [14]:

os.chdir('F:/Environmental Baseline Data/Version 3/Data/PDF/')

In [15]:
import glob
len(glob.glob('F:/Environmental Baseline Data/Version 3/Data/PDF/*.pdf'))

1076

In [18]:
#downloads all ESA PDFs from list
for x in download_ESA_links[1071:]:
    try:
        urlretrieve(x,download_ESA_filename[pd.Index(download_ESA_links).get_loc(x)]+'.pdf')
    except:
        print("Failed")


('A4E6Y1.pdf', <http.client.HTTPMessage at 0x259d072c588>)

('A4E6Y2.pdf', <http.client.HTTPMessage at 0x259d072c080>)

('A4E6Y3.pdf', <http.client.HTTPMessage at 0x259d072cf98>)

('A4E6Y4.pdf', <http.client.HTTPMessage at 0x259d072c1d0>)

('A4E6Y5.pdf', <http.client.HTTPMessage at 0x259d072c908>)

...

('A1C3Z1.pdf', <http.client.HTTPMessage at 0x259d04140f0>)

('A1C3Z2.pdf', <http.client.HTTPMessage at 0x259d0502748>)

('A1C3Z3.pdf', <http.client.HTTPMessage at 0x259d0502b38>)

('A1C3Z4.pdf', <http.client.HTTPMessage at 0x259d0502c50>)

('A1C3Z5.pdf', <http.client.HTTPMessage at 0x259d05022b0>)

In [137]:
#regular expressions to parse out application date and hearing order 
application_date = df['Application title'].str.extract('(\d\d\d\d-\d\d-\d\d)')
df.insert(7, 'application_date', application_date)

hearing_order = df['Application title'].str.extract(r'(\D\D-\d(.+?)\d\d\d\d)')
hearing_order = hearing_order.drop([1], axis=1)
df.insert(8, 'hearing_order', hearing_order)

In [141]:
#check to ensure regex worked
hearing_order.isnull().values.any()
application_date.isnull().values.any()

False

False

In [139]:
df.shape
df.columns
df['Application title'].nunique()

(1331, 11)

Index(['file_name', 'postal', 'hrefs', 'folder', 'Folder link', 'Section',
       'Application title', 'application_date', 'hearing_order',
       'link to folder with ESA', 'Company Name'],
      dtype='object')

36

In [140]:
df.to_csv('ESA_Index1.csv')